In [2]:
import sys 
import os
import pandas as pd
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from utils.utils import save_experiment, evaluate_model_on_parquet, grid_search_model, train_and_evaluate_non_linear_svm
from configs.config_local import DATASET_PATH_FEATURES, ITW_DATASET_PATH, MODELS_PATH
import joblib


In [5]:
train_data_path = os.path.join(DATASET_PATH_FEATURES, "training_features_20_128_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH_FEATURES, "testing_features_20_128_256_128.parquet")
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_20_128_256_128.parquet")

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
df_itw = pd.read_parquet(itw_test_data_path)


#print(df.head())
print(df.shape)
df.dropna(inplace=True)
#print(df.head())
print(df.shape)


#print(df_test.head())
print(df_test.shape)
df_test.dropna(inplace=True)
#print(df_test.head())
print(df_test.shape)

#print(df_itw.head())
print(df_itw.shape)
df_itw.dropna(inplace=True)
#print(df_itw.head())
print(df_itw.shape)


(53868, 392)
(53864, 392)
(4634, 392)
(4634, 392)
(31779, 392)
(31526, 392)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC()),
])

param_grid = {
    "svm__kernel": ["rbf"],
    "svm__C": [0.1, 1, 10, 100, 1000],  # maybe stop at 100 for RBF
    "svm__gamma": ["scale", 0.01, 0.001],
    "svm__class_weight": [None, "balanced"],
}

train_data_path = os.path.join(DATASET_PATH_FEATURES, "training_features_20_128_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH_FEATURES, "testing_features_20_128_256_128.parquet")



best_model, metrics, best_params_, metadata, feature_names = grid_search_model(
    model,
    param_grid,
    train_data_path,
    test_data_path,
    scoring= "f1_macro",
    cv = 5,
    n_jobs= 1,
    verbose = 2,
)

print(best_params_)
print(metrics)


save_experiment(
    model=best_model,
    metrics=metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments",  "rbf_svm",),
    model_params=best_params_,
    feature_names=feature_names,
    metadata_extra=metadata,
)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END svm__C=0.1, svm__class_weight=None, svm__gamma=scale, svm__kernel=rbf; total time=   1.2s
[CV] END svm__C=0.1, svm__class_weight=None, svm__gamma=scale, svm__kernel=rbf; total time=   0.5s
[CV] END svm__C=0.1, svm__class_weight=None, svm__gamma=scale, svm__kernel=rbf; total time=   0.5s
[CV] END svm__C=0.1, svm__class_weight=None, svm__gamma=scale, svm__kernel=rbf; total time=   0.5s
[CV] END svm__C=0.1, svm__class_weight=None, svm__gamma=scale, svm__kernel=rbf; total time=   0.4s


In [ ]:
train_data_path = os.path.join(DATASET_PATH_FEATURES, "training_features_20_128_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH_FEATURES, "testing_features_20_128_256_128.parquet")

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

svc_params = {
            "kernel": "rbf",
            "C": 1.0,
            "class_weight": {0: 1, 1: 5},
            "max_iter": 20000,
            "random_state": 42,
        }
pipeline, metrics, svc_params, feature_names, metadata_extra = train_and_evaluate_non_linear_svm(train_data_path, test_data_path, svc_params)
print(metrics)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -622.954895   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -579.459290   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -666.195618   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -658.100586   
4  real  file10000.wav_16k.wav_norm.wav_mono.wav_silenc...  -603.779663   

   mfcc_std_0  mfcc_mean_1  mfcc_std_1  mfcc_mean_2  mfcc_std_2  mfcc_mean_3  \
0   78.503250    -1.842725   53.011627   -22.277306   35.032856    27.215727   
1   53.074417    -8.681670   44.406063   -14.242689   24.373398    46.426991   
2   76.873924    -4.181276   57.543621    10.106292   22.276455    28.408127   
3   77.856445    -0.066092   59.765896    -6.204688   42.900917    22.124472   
4   59.765564     0.579489   65.219322   -15.271199   50.885433    20.162930   

   mfcc_std_3  ...  mel_spectrogram_mean_123  mel_spectrogram_std_12

Test Best Model to ITW

In [4]:
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_20_128_256_128.parquet")

In [5]:

metrics, metadata_extra = evaluate_model_on_parquet(pipeline, itw_test_data_path)
print(metrics)

{'accuracy': 0.6691302417052591, 'precision': 0.7522255192878339, 'recall': 0.17199558985667035, 'f1_macro': 0.5325954202232782, 'roc_auc': 0.7518715680588165}


In [6]:
train_data_path = os.path.join(DATASET_PATH_FEATURES, "training_features_no_std_20_128_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH_FEATURES, "testing_features_no_std_20_128_256_128.parquet")

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

svc_params = {
            "kernel": "rbf",
            "C": 1.0,
            "class_weight": {0: 1, 1: 5},
            "max_iter": 20000,
            "random_state": 42,
        }
pipeline, metrics, svc_params, feature_names, metadata_extra = train_and_evaluate_non_linear_svm(train_data_path, test_data_path, svc_params)
print(metrics)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -611.930420   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -605.008606   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -597.774414   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -601.991211   
4  real  file1001.wav_16k.wav_norm.wav_mono.wav_silence...  -624.717590   

   mfcc_mean_1  mfcc_mean_2  mfcc_mean_3  mfcc_mean_4  mfcc_mean_5  \
0    12.266665   -12.647933    22.397125   -18.097271     6.636819   
1    -0.973098   -12.821069    38.964752   -17.512388    -5.345759   
2     7.211399   -16.026072    50.700493   -41.957108    -4.920807   
3   -23.718582    -9.058393    42.911995   -23.795502    -0.905054   
4   -23.051550   -12.977422    38.695904   -18.944435    -0.490484   

   mfcc_mean_6  mfcc_mean_7  ...  mel_spectrogram_mean_118  \
0   -13.808917     1.950459  ...                  0.104165   
1   

In [7]:
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_no_std_20_128_256_128.parquet")

In [8]:

metrics, metadata_extra = evaluate_model_on_parquet(pipeline, itw_test_data_path)
print(metrics)

{'accuracy': 0.7092558523123771, 'precision': 0.664124046517444, 'recall': 0.4504282927656687, 'f1_macro': 0.6624639587542147, 'roc_auc': 0.7872281308888013}
